In [1]:
%pip install -q --upgrade sentence-transformers
%pip install -q --upgrade faiss-cpu
%pip install -q --upgrade faiss-gpu
%pip install -q --upgrade scikit-learn
%pip install -q --upgrade pandas
%pip install -q --upgrade numpy
%pip install -q --upgrade matplotlib
import pandas as pd
from sklearn.ensemble import IsolationForest
import random
import warnings
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.model_selection import train_test_split
import os
import faiss
from sklearn.neighbors import LocalOutlierFactor

warnings.filterwarnings("ignore")
ANOMALY_LABELS = ["is_fraud"]
BINARY_LABEL = "is_fraud"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


c:\Users\orric\python_projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# Choose between OpenAI or AzureOpenAI
USE_AZURE = False  # Set to True if running experiments at work

if USE_AZURE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        azure_endpoint="YOUR_AZURE_ENDPOINT",
        api_key="YOUR_AZURE_API_KEY",
        api_version="DATE"
    )
    deployment_name = "DEPLOYMENT_NAME"  # Replace with your actual deployment name
else:
    import openai
    client = openai.OpenAI(api_key="YOUR CHATGPT API KEY")  # Replace with your actual key or set as env var
    deployment_name = "gpt-4o"


## 📦 Dataset Loader and Preprocessing

This section defines functions for:
- Loading different fraud-related datasets
- Splitting into train/test sets
- Balancing the data for anomaly detection (important since fraud is rare!)


In [6]:
# === Dataset Loader + Test ===

def load_dataset(name):
    """
    Loads dataset based on name and updates anomaly label type.
    Returns: dataframe, domain name
    """
    global ANOMALY_LABELS

    if name == "credit":
        df = pd.read_csv("creditcard.csv", index_col=0)
        domain = "credit"
        ANOMALY_LABELS = ["Class"]
    elif name == "insurance":
        df = pd.read_csv("insurance_claim.csv", index_col=0)
        domain = "insurance"
        ANOMALY_LABELS = ["insurance_claim"]
    elif name == "fraudtest":
        df = pd.read_csv("fraudTest.csv", index_col=0)
        domain = "fraudtest"
        ANOMALY_LABELS = ["is_fraud"]
    else:
        raise ValueError(f"❌ Unknown dataset: {name}")
    
    print(f"📄 Loaded dataset: '{name}' | Domain: {domain} | Shape: {df.shape}")
    print(f"🔎 Anomaly label column: {ANOMALY_LABELS[0]}")
    return df, domain

# === Run a test load on "fraudtest" dataset ===
df, domain = load_dataset("fraudtest")

# Show a preview of the data
print("\n🔍 Sample data:")
display(df.head())


📄 Loaded dataset: 'fraudtest' | Domain: fraudtest | Shape: (555719, 22)
🔎 Anomaly label column: is_fraud

🔍 Sample data:


trans_date_trans_time            cc_num  \
0   2020-06-21 12:14:25  2291163933867244   
1   2020-06-21 12:14:33  3573030041201292   
2   2020-06-21 12:14:53  3598215285024754   
3   2020-06-21 12:15:15  3591919803438423   
4   2020-06-21 12:15:17  3526826139003047   

                               merchant        category    amt   first  \
0                 fraud_Kirlin and Sons   personal_care   2.86    Jeff   
1                  fraud_Sporer-Keebler   personal_care  29.84  Joanne   
2  fraud_Swaniawski, Nitzsche and Welch  health_fitness  41.28  Ashley   
3                     fraud_Haley Group        misc_pos  60.05   Brian   
4                 fraud_Johnston-Casper          travel   3.19  Nathan   

       last gender                       street        city  ...      lat  \
0   Elliott      M            351 Darlene Green    Columbia  ...  33.9659   
1  Williams      F             3638 Marsh Union     Altonah  ...  40.3207   
2     Lopez      F         9333 Valentine Point    Bellmore  ...  40.6729   
3  Williams      M  32941 Krystal Mill Apt. 552  Titusville  ...  28.5697   
4    Massey      M     5783 Evan Roads Apt. 465    Falmouth  ...  44.2529   

       long  city_pop                     job         dob  \
0  -80.9355    333497     Mechanical engineer  1968-03-19   
1 -110.4360       302  Sales professional, IT  1990-01-17   
2  -73.5365     34496       Librarian, public  1970-10-21   
3  -80.8191     54767            Set designer  1987-07-25   
4  -85.0170      1126      Furniture designer  1955-07-06   

                          trans_num   unix_time  merch_lat  merch_long  \
0  2da90c7d74bd46a0caf3777415b3ebd3  1371816865  33.986391  -81.200714   
1  324cc204407e99f51b0d6ca0055005e7  1371816873  39.450498 -109.960431   
2  c81755dbbbea9d5c77f094348a7579be  1371816893  40.495810  -74.196111   
3  2159175b9efe66dc301f149d3d5abf8c  1371816915  28.812398  -80.883061   
4  57ff021bd3f328f8738bb535c302a31b  1371816917  44.959148  -85.884734   

   is_fraud  
0         0  
1         0  
2         0  
3         0  
4         0  

[5 rows x 22 columns]

In [23]:
# === Clean up merchant names ===

if "merchant" in df.columns:
    df["merchant"] = df["merchant"].str.replace("^fraud_", "", regex=True)
    print("🧽 Cleaned 'merchant' column — removed misleading 'fraud_' prefix.")
else:
    print("⚠️ No 'merchant' column found in the dataset.")

# Show a few example merchant names
print("\n🛍️ Sample merchant names:")
print(df["merchant"].dropna().unique()[:10])


🧽 Cleaned 'merchant' column — removed misleading 'fraud_' prefix.

🛍️ Sample merchant names:
['Kirlin and Sons' 'Sporer-Keebler' 'Swaniawski, Nitzsche and Welch'
 'Haley Group' 'Johnston-Casper' 'Daugherty LLC' 'Romaguera Ltd'
 'Reichel LLC' 'Goyette, Howell and Collier' 'Kilback Group']


In [24]:
# === Define and run preprocessing function ===

def preprocess_data_train_test(df: pd.DataFrame, test_size: float = 0.3, random_state: int = 42):
    """
    Splits the data into train/test sets and balances each subset.
    Balancing is done by undersampling the majority class.
    """
    print("🧪 Splitting dataset into train/test...")
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state)
    print(f"→ Train shape: {train_df.shape}, Test shape: {test_df.shape}")

    def balance_data(df_subset: pd.DataFrame) -> pd.DataFrame:
        print(f"⚖️ Balancing on: {ANOMALY_LABELS[0]}")
        fraud = df_subset[df_subset[ANOMALY_LABELS[0]] == 1]
        non_fraud = df_subset[df_subset[ANOMALY_LABELS[0]] == 0]

        if len(fraud) == 0:
            raise ValueError("❌ No fraud samples found — cannot balance.")
        if len(non_fraud) < len(fraud):
            raise ValueError("❌ Not enough non-fraud samples to balance.")

        non_fraud_balanced = non_fraud.sample(n=len(fraud), random_state=random_state)
        balanced_df = pd.concat([fraud, non_fraud_balanced]).sample(frac=1, random_state=random_state).reset_index(drop=True)

        print(f"✔️ Balanced to {len(balanced_df)} rows: {len(fraud)} fraud / {len(non_fraud_balanced)} non-fraud")
        return balanced_df

    return balance_data(train_df), balance_data(test_df)

# === Apply preprocessing on the loaded dataset ===
train_df, test_df = preprocess_data_train_test(df)

# Show class distribution
print("\n📊 Class distribution in TRAIN:")
print(train_df[ANOMALY_LABELS[0]].value_counts())

print("\n📊 Class distribution in TEST:")
print(test_df[ANOMALY_LABELS[0]].value_counts())

# Optional: peek at the first few rows
print("\n🔍 Sample from balanced train set:")
display(train_df.head())


🧪 Splitting dataset into train/test...
→ Train shape: (389003, 22), Test shape: (166716, 22)
⚖️ Balancing on: is_fraud
✔️ Balanced to 3010 rows: 1505 fraud / 1505 non-fraud
⚖️ Balancing on: is_fraud
✔️ Balanced to 1280 rows: 640 fraud / 640 non-fraud

📊 Class distribution in TRAIN:
is_fraud
0    1505
1    1505
Name: count, dtype: int64

📊 Class distribution in TEST:
is_fraud
0    640
1    640
Name: count, dtype: int64

🔍 Sample from balanced train set:


trans_date_trans_time            cc_num                     merchant  \
0   2020-12-17 19:43:33  3508104404056005  Eichmann, Hayes and Treutel   
1   2020-07-19 22:21:21  3534330126107879             Heaney-Marquardt   
2   2020-08-17 16:08:49  4951647200979051                   Jewess LLC   
3   2020-11-21 23:21:08    30235268718158   Swaniawski, Lowe and Robel   
4   2020-11-25 23:21:38    30546453310167              Turner and Sons   

        category      amt     first    last gender  \
0         travel     5.04   Michael  Fisher      M   
1  entertainment   503.53   Jeffrey   Smith      M   
2   shopping_pos     9.24  Kimberly  Miller      F   
3   shopping_pos   641.44   Kenneth   Doyle      M   
4   shopping_pos  1033.33  Nicholas    Tran      M   

                       street           city  ...      lat      long  \
0            401 Escobar Port      East Troy  ...  42.8035  -88.4092   
1     713 Scott Pike Apt. 712        Bridger  ...  45.2857 -108.9082   
2        75533 Tamara Valleys          Logan  ...  37.9943  -88.9417   
3              8614 Reed Glen  West Harrison  ...  41.0592  -73.7395   
4  672 Randall Burg Suite 551        Belmont  ...  43.4512  -71.4890   

   city_pop                                      job         dob  \
0      9679                     Engineer, production  1991-04-11   
1      1446                  Chartered loss adjuster  1978-01-15   
2       324  Scientist, research (physical sciences)  1976-06-15   
3     11250                            Lexicographer  1977-08-16   
4      7430            Scientist, research (medical)  1955-01-13   

                          trans_num   unix_time  merch_lat  merch_long  \
0  71e28a3a8245558e00651c29bf780df2  1387309413  42.901649  -88.572310   
1  a4b226b750ab2e04f80c024834977b15  1374272481  44.557191 -107.920422   
2  f5100a2a767a6390eb3d3c02f6dc2de2  1376755729  38.846076  -89.075456   
3  7bcac797f7acba84130e417dadaf78cd  1385076068  40.124429  -73.783699   
4  51126a17579a482a887addd137a1ea96  1385421698  42.851048  -72.418639   

   is_fraud  
0         0  
1         1  
2         0  
3         1  
4         1  

[5 rows x 22 columns]

# 🧬 Feature Embedding

This section transforms structured data into dense numeric vectors (embeddings) that can be compared and clustered.

We:
- Identify numeric vs. textual columns
- Scale numeric data using `StandardScaler`
- Embed any text columns using a pretrained transformer (e.g. `all-MiniLM-L6-v2`)
- Combine everything into one high-dimensional feature matrix

This enables us to later compute similarity between transactions and find meaningful precedent cases.


In [25]:
# === Define and run feature embedding preprocessing ===

def preprocess_data_LP(train_df):
    """
    Prepares training embeddings:
    - Scales numerical features
    - Embeds textual features using SentenceTransformer
    - Returns combined embedding matrix + metadata
    """
    train_df_features = train_df.drop(columns=ANOMALY_LABELS, errors="ignore")

    # Identify feature types
    numeric_cols = train_df_features.select_dtypes(include=["number"]).columns.tolist()
    text_cols = [col for col in train_df_features.columns if col not in numeric_cols]

    print(f"🔢 Numeric columns: {numeric_cols}")
    print(f"📝 Text columns: {text_cols if text_cols else 'None'}")

    # Scale numeric
    scaler = StandardScaler()
    numeric_train = scaler.fit_transform(train_df_features[numeric_cols])

    # Embed text (if any)
    if text_cols:
        model = SentenceTransformer("all-MiniLM-L6-v2")
        train_df_features["combined_text"] = train_df_features[text_cols].astype(str).agg(" ".join, axis=1)
        print(f"✍️ Sample combined text:\n{train_df_features['combined_text'].iloc[0][:150]}...")

        text_train = model.encode(train_df_features["combined_text"].tolist(), show_progress_bar=True)
        train_embeddings = np.hstack([numeric_train, text_train])
    else:
        model = None
        train_embeddings = numeric_train

    print(f"\n✅ Final embedding shape: {train_embeddings.shape}")
    print("🔍 Sample row from embeddings:")
    print(train_embeddings[np.random.choice(train_embeddings.shape[0])])

    return train_embeddings, scaler, model, numeric_cols, text_cols

# === Run on the balanced training dataframe ===
train_embeddings, scaler, model, numeric_cols, text_cols = preprocess_data_LP(train_df)


🔢 Numeric columns: ['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long']
📝 Text columns: ['trans_date_trans_time', 'merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num']
✍️ Sample combined text:
2020-12-17 19:43:33 Eichmann, Hayes and Treutel travel Michael Fisher M 401 Escobar Port East Troy WI Engineer, production 1991-04-11 71e28a3a8245558e...


Batches: 100%|██████████| 95/95 [00:18<00:00,  5.25it/s]



✅ Final embedding shape: (3010, 393)
🔍 Sample row from embeddings:
[-3.23816868e-01 -6.68385069e-01 -8.64498647e-01 -1.12622345e-01
  6.23578046e-01 -2.97526035e-01  8.87005336e-01  7.37133213e-02
  6.63924512e-01 -4.69736531e-02 -2.32779570e-02  6.45417273e-02
  3.40940543e-02  6.19966909e-02  1.33547559e-01  9.46907997e-02
 -3.71835083e-02  9.81913786e-03 -4.68247645e-02 -4.17864583e-02
 -5.87417334e-02 -4.72383350e-02  1.77896451e-02 -8.70690942e-02
  1.81512013e-02 -1.70861054e-02 -5.57014085e-02 -2.69274358e-02
  6.50664838e-03 -3.28589156e-02  4.21820357e-02  1.70272123e-02
  7.72716850e-02 -7.17371628e-02  2.79254559e-02 -4.78307009e-02
 -5.39575377e-03  6.08075038e-03 -3.39351408e-02 -1.28905065e-02
  1.55354170e-02 -9.71417204e-02  9.07501727e-02  9.89679247e-02
  1.22597795e-02  4.77721123e-03  2.96350494e-02 -4.45326790e-02
  2.84891445e-02 -4.40967008e-02 -1.60931587e-01 -3.79978083e-02
  5.30357007e-03 -5.92592396e-02 -1.10727139e-02 -2.30380408e-02
 -5.06025888e-02 -2.22

In [26]:
# === Define and run FAISS index creation ===

def create_index(embeddings):
    """
    Builds a FAISS index for fast nearest neighbor search.
    """
    d = embeddings.shape[1]  # dimensionality of embeddings
    index = faiss.IndexFlatL2(d)
    index.add(embeddings.astype(np.float32))
    print(f"📚 FAISS index built with dimension: {d} and {embeddings.shape[0]} entries.")
    return index

# === Build index from our train embeddings ===
index = create_index(train_embeddings)

# === Full nearest neighbor preview with all fields ===
sample_id = np.random.randint(len(train_embeddings))
query_embedding = train_embeddings[sample_id:sample_id+1].astype(np.float32)
D, I = index.search(query_embedding, k=5)

print(f"🔍 Nearest neighbors to training row #{sample_id}")
print("🧾 Transaction under investigation:")
display(train_df.iloc[[sample_id]])

# Pull and annotate full neighbor rows
neighbors_df = train_df.iloc[I[0]].copy()
neighbors_df["legal_distance"] = D[0]
neighbors_df["neighbor_index"] = I[0]

# Reorder to put metadata first
cols = ["neighbor_index", "legal_distance"] + [col for col in neighbors_df.columns if col not in ["neighbor_index", "legal_distance"]]
neighbors_df = neighbors_df[cols]

print("\n📚 Nearest neighbor transactions (full details):")
display(neighbors_df)



📚 FAISS index built with dimension: 393 and 3010 entries.
🔍 Nearest neighbors to training row #2369
🧾 Transaction under investigation:


trans_date_trans_time            cc_num    merchant      category  \
2369   2020-06-26 23:18:46  6544734391390261  Schumm PLC  shopping_net   

         amt first   last gender             street      city  ...    lat  \
2369  949.88  John  Myers      M  701 Wilson Divide  Highland  ...  41.55   

         long  city_pop                    job         dob  \
2369 -87.4569     23727  Community arts worker  1982-02-08   

                             trans_num   unix_time  merch_lat  merch_long  \
2369  913d395d3dfc5d93fdf25dc9f7870653  1372288726  41.618135  -87.554747   

      is_fraud  
2369         1  

[1 rows x 22 columns]


📚 Nearest neighbor transactions (full details):


neighbor_index  legal_distance trans_date_trans_time            cc_num  \
2369            2369        0.000000   2020-06-26 23:18:46  6544734391390261   
2704            2704        0.270126   2020-06-26 23:38:10  6544734391390261   
443              443        0.659173   2020-06-27 02:06:49  6544734391390261   
2522            2522        1.478805   2020-06-21 22:32:22  6564459919350820   
891              891        1.656684   2020-07-25 21:43:42    38014427445058   

                          merchant      category      amt    first    last  \
2369                    Schumm PLC  shopping_net   949.88     John   Myers   
2704               Kerluke-Abshire  shopping_net  1025.61     John   Myers   
443   Tillman, Fritsch and Schmitt      misc_net   764.71     John   Myers   
2522   Rodriguez, Yost and Jenkins      misc_net   780.52  Douglas  Willis   
891                       Jast Ltd  shopping_net   974.65    Sonya  Jensen   

     gender  ...      lat     long city_pop                              job  \
2369      M  ...  41.5500 -87.4569    23727            Community arts worker   
2704      M  ...  41.5500 -87.4569    23727            Community arts worker   
443       M  ...  41.5500 -87.4569    23727            Community arts worker   
2522      M  ...  42.5545 -90.3508     1306         Public relations officer   
891       F  ...  43.8967 -89.8219     3508  Sport and exercise psychologist   

             dob                         trans_num   unix_time  merch_lat  \
2369  1982-02-08  913d395d3dfc5d93fdf25dc9f7870653  1372288726  41.618135   
2704  1982-02-08  92cb4901841afb9c374df14d36dfac38  1372289890  41.802631   
443   1982-02-08  edcfe3e9c8859adcb31bba8bdb90286d  1372298809  41.870903   
2522  1958-09-10  ab4b379d2c0c9c667d46508d4e126d72  1371853942  42.461127   
891   1946-08-11  59150de6c122d18f500f13f31cff1078  1374788622  43.031695   

     merch_long is_fraud  
2369 -87.554747        1  
2704 -88.389592        1  
443  -87.172741        1  
2522 -91.147148        1  
891  -89.149317        1  

[5 rows x 24 columns]

In [27]:
# === Define and run test row embedding ===

def embed_test_sample(test_row, scaler, model, numeric_cols, text_cols):
    """
    Embeds a single test row using the same numeric scaling and text model as training.
    Returns a 2D array: shape (1, total_feature_dim)
    """
    test_df = test_row.to_frame().T  # Make it a 1-row DataFrame

    # Drop label column if present
    test_features = test_df.drop(columns=ANOMALY_LABELS, errors="ignore")

    # === Numeric part ===
    numeric_test = scaler.transform(test_features[numeric_cols])

    # === Text part ===
    if text_cols and model is not None:
        test_features["combined_text"] = test_features[text_cols].astype(str).agg(" ".join, axis=1)
        text_test = model.encode(test_features["combined_text"].tolist())
        test_embedding = np.hstack([numeric_test, text_test])
    else:
        test_embedding = numeric_test

    # print(f"✅ Test row embedded! Final shape: {test_embedding.shape}")
    return test_embedding

# === Try it out on a sample test row ===
sample_test_row = test_df.sample(1).iloc[0]
test_embedding = embed_test_sample(sample_test_row, scaler, model, numeric_cols, text_cols)

# Peek at it
print("\n🔍 Sample test embedding (truncated):")
print(np.round(test_embedding[0][:10], 4), "...")  # just the first 10 dims



🔍 Sample test embedding (truncated):
[-0.3224 -0.7903  1.4656 -0.5109 -1.119  -0.299   0.2967 -0.6078 -1.0531
 -0.0339] ...


In [28]:
# === Define and run nearest neighbor finder ===

def find_nearest_neighbors(test_row, train_df, index, scaler, model, numeric_cols, text_cols, k=5):
    """
    Embeds the test row, queries the FAISS index, and returns the top-k nearest training rows.
    """
    test_embedding = embed_test_sample(test_row, scaler, model, numeric_cols, text_cols)
    D, I = index.search(test_embedding.astype(np.float32), k)

    neighbors = train_df.iloc[I[0]].copy()
    neighbors["legal_distance"] = D[0]
    neighbors["neighbor_index"] = I[0]

    # Reorder for clarity
    cols = ["legal_distance"] + [col for col in neighbors.columns if col not in ["neighbor_index", "legal_distance"]]
    return neighbors[cols]

# === Select and display a new sample test row ===
sample_test_row = test_df.sample(1).iloc[0]
print("🕵️ Transaction under investigation:")
display(sample_test_row.to_frame().T)

# === Get and display its nearest neighbors ===
neighbors_df = find_nearest_neighbors(sample_test_row, train_df, index, scaler, model, numeric_cols, text_cols, k=5)
print("📚 Most similar precedent transactions:")
display(neighbors_df)


🕵️ Transaction under investigation:


trans_date_trans_time            cc_num        merchant   category  \
635   2020-07-19 14:01:25  6538441737335434  Schimmel-Olson  kids_pets   

        amt first    last gender           street          city  ...      lat  \
635  154.33  Gina  Grimes      F  444 Robert Mews  Clarks Mills  ...  41.3851   

        long city_pop             job         dob  \
635 -80.1752      606  Energy manager  1997-09-22   

                            trans_num   unix_time  merch_lat merch_long  \
635  08723bde83f058d7aabc0b2cd5e7c36d  1374242485  40.575345 -79.417025   

    is_fraud  
635        0  

[1 rows x 22 columns]

📚 Most similar precedent transactions:


legal_distance trans_date_trans_time            cc_num  \
402         0.899750   2020-07-27 21:32:45  3543591270174051   
194         1.109416   2020-07-27 11:36:15    30428204673351   
2118        1.149924   2020-08-03 22:24:36  3529597810116815   
605         1.156514   2020-07-27 12:41:32   213186334552049   
1241        1.158025   2020-07-30 02:36:24  4149238353975790   

                          merchant       category     amt     first  \
402                 Crona and Sons      kids_pets   57.80  Margaret   
194   Quitzon, Green and Bashirian   shopping_pos  148.81      Greg   
2118               Leffler-Goldner  personal_care   64.99    Pamela   
605    Haley, Jewess and Bechtelar   shopping_pos    2.82    Andrea   
1241                 Hills-Witting   shopping_net  114.88    Tanner   

            last gender                         street  ...      lat     long  \
402          Lam      F    6911 Nicholas Keys Apt. 237  ...  40.4603 -79.0097   
194   Villarreal      M             2851 Foster Shoals  ...  40.3087 -76.2963   
2118      Curtis      F  8969 Fuentes Station Apt. 291  ...  40.4679 -77.8967   
605      Perkins      F           37426 Olson Villages  ...  41.8117 -79.2643   
1241     Carroll      M                494 Burke Ports  ...  40.1008 -80.0652   

      city_pop                      job         dob  \
402        922      Early years teacher  1972-10-04   
194        823                   Gaffer  1982-07-02   
2118      1334            Archaeologist  1986-12-17   
605        528  Chief Marketing Officer  1972-09-13   
1241       632                   Dealer  1989-04-08   

                             trans_num   unix_time  merch_lat merch_long  \
402   42a04d1b6748c1654fcac8fe288ab5a7  1374960765  40.235226 -78.680842   
194   aad9aab0e0f0cbae1043ee9a4a715457  1374924975  39.808787 -76.179351   
2118  be12ffb105de4bc05db57d822192e73a  1375568676  39.881263 -78.027545   
605   70c3231ede9ee6a82e1174ffb3c7b82b  1374928892  41.705691 -78.354888   
1241  202d5aeb0c65561a243359b6447e954c  1375151784  40.803875 -79.868393   

      is_fraud  
402          0  
194          0  
2118         0  
605          0  
1241         0  

[5 rows x 23 columns]

In [29]:
# === Define updated neighbor formatter with clean sequential IDs ===

def format_neighbors_generic(df_neighbors, k=5):
    """
    Formats neighbors as readable entries, numbered from 1 to k,
    ignoring original dataframe indices.
    """
    output_lines = []
    selected = df_neighbors.head(k).reset_index(drop=True)  # drop original index

    for i, row in selected.iterrows():
        parts = []
        for col in df_neighbors.columns:
            if col == "legal_distance":
                parts.append(f"Distance: {row[col]:.3f}")
            elif isinstance(row[col], (int, float)):
                parts.append(f"{col}: {row[col]}")
            else:
                val = str(row[col])
                if len(val) > 100:
                    val = val[:100] + "..."
                parts.append(f"{col}: {val}")
        output_lines.append(f"Neighbor {i + 1}:\n" + "; ".join(parts) + "\n")

    return "\n".join(output_lines)


# === Format the neighbors we found previously ===
formatted_neighbors = format_neighbors_generic(neighbors_df)

print("📜 Precedent report:")
print(formatted_neighbors)

📜 Precedent report:
Neighbor 1:
Distance: 0.900; trans_date_trans_time: 2020-07-27 21:32:45; cc_num: 3543591270174051; merchant: Crona and Sons; category: kids_pets; amt: 57.8; first: Margaret; last: Lam; gender: F; street: 6911 Nicholas Keys Apt. 237; city: Armagh; state: PA; zip: 15920; lat: 40.4603; long: -79.0097; city_pop: 922; job: Early years teacher; dob: 1972-10-04; trans_num: 42a04d1b6748c1654fcac8fe288ab5a7; unix_time: 1374960765; merch_lat: 40.235226; merch_long: -78.680842; is_fraud: 0

Neighbor 2:
Distance: 1.109; trans_date_trans_time: 2020-07-27 11:36:15; cc_num: 30428204673351; merchant: Quitzon, Green and Bashirian; category: shopping_pos; amt: 148.81; first: Greg; last: Villarreal; gender: M; street: 2851 Foster Shoals; city: Schaefferstown; state: PA; zip: 17088; lat: 40.3087; long: -76.2963; city_pop: 823; job: Gaffer; dob: 1982-07-02; trans_num: aad9aab0e0f0cbae1043ee9a4a715457; unix_time: 1374924975; merch_lat: 39.808787; merch_long: -76.179351; is_fraud: 0

Neig

# 🔬 The Fact-Checker’s Toolkit

This section defines a flexible system for anomaly detection on individual transactions.

Depending on the dataset domain (e.g. **credit**, **insurance**, or **fraudtest**), the fact-checker uses a specialized toolkit of interpretable statistical checks:

- **Z-Score anomalies** — highlights fields that deviate from the mean by more than 3 standard deviations.
- **IQR Outliers** — identifies fields that lie far outside the interquartile range.
- **Benford’s Law** — flags suspicious numeric distributions (e.g., amounts).
- **Heuristic flags** — dataset-specific rules (e.g., late-night + high-amount for credit).

The result is a concise, judge-readable report that explains what (if anything) is abnormal about a transaction.

This system is modular and extensible. As we evaluate new datasets, we may need to adjust or extend the toolkit for that domain.


In [30]:
# === Define Fact Checker Agent with detailed field info ===

def generate_fact_checker_report(row, df_reference, domain):
    """
    Applies multiple anomaly detection tools to a transaction.
    Returns a human-readable summary with specific field names.
    """
    facts = []
    numeric_cols = df_reference.select_dtypes(include=[np.number]).columns.tolist()
    row_numeric = row[numeric_cols]
    df_numeric = df_reference[numeric_cols]

    def z_score_anomaly_check(row, df, threshold=3):
        z_scores = ((row - df.mean()) / df.std()).abs()
        flagged = z_scores[z_scores > threshold]
        return flagged.to_dict()

    def benford_law_check(amount):
        if amount <= 0:
            return False
        digits = [int(str(int(amount))[0])]
        digit_counts = [0] * 9
        digit_counts[digits[0] - 1] += 1
        benford_expected = [0.301, 0.176, 0.125, 0.097, 0.079, 0.067, 0.058, 0.051, 0.046]
        chi_squared = sum(((digit_counts[i] - benford_expected[i]) ** 2) / benford_expected[i] for i in range(9))
        return chi_squared > 15.51

    def outlier_detection_flags(row, df):
        """
        Returns a list of columns that are IQR outliers
        """
        q1 = df.quantile(0.25)
        q3 = df.quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        outlier_flags = ((row < lower) | (row > upper))
        return outlier_flags[outlier_flags].index.tolist()

    def heuristic_check(row):
        if "Time" in row and "Amount" in row:
            is_odd_time = (row["Time"] % 86400) > 79200 or (row["Time"] % 86400) < 18000
            is_large_amt = row["Amount"] > 200
            return is_odd_time and is_large_amt
        return False

    # === Domain-specific logic ===
    z_anomalies, outlier_fields, benford_flag, heuristic_flag = {}, [], False, False

    if domain == "insurance":
        relevant = [col for col in ["age", "bmi", "children", "charges"] if col in df_numeric.columns]
        z_anomalies = z_score_anomaly_check(row[relevant], df_reference[relevant])
        outlier_fields = outlier_detection_flags(row[relevant], df_reference[relevant])
        benford_flag = benford_law_check(row.get("charges", 0))

    elif domain == "credit":
        z_anomalies = z_score_anomaly_check(row_numeric, df_numeric)
        outlier_fields = outlier_detection_flags(row_numeric, df_numeric)
        heuristic_flag = heuristic_check(row)

    elif domain == "fraudtest":
        z_anomalies = z_score_anomaly_check(row_numeric, df_numeric)
        outlier_fields = outlier_detection_flags(row_numeric, df_numeric)
        benford_flag = benford_law_check(row.get("amt", 0))

    else:
        return "⚠️ Fact Checker: Unrecognized domain — no checks applied."

    # === Compile report ===
    if z_anomalies:
        facts.append(f"⚠️ Z-score anomaly in: {', '.join(z_anomalies.keys())}")
    if outlier_fields:
        facts.append(f"⚠️ IQR outlier in: {', '.join(outlier_fields)}")
    if domain == "credit" and heuristic_flag:
        facts.append("⚠️ Heuristic flagged: odd time + high amount.")
    if domain != "credit" and benford_flag:
        facts.append("⚠️ Benford's Law violated for amount.")

    return "✅ No anomalies found." if not facts else "📋 Fact Checker Report:\n" + "\n".join(facts)

# === Run on a new sample row ===
sample_test_row = test_df.sample(1).iloc[0]

print("🧾 Sample transaction under review:")
display(sample_test_row.to_frame().T)

fact_checker_result = generate_fact_checker_report(sample_test_row, train_df, domain)

print("\n🕵️ Fact Checker result:")
print(fact_checker_result)


🧾 Sample transaction under review:


trans_date_trans_time            cc_num     merchant category   amt first  \
306   2020-09-07 12:27:25  2264937662466770  Kutch-Ferry     home  15.6  Juan   

        last gender                        street         city  ...      lat  \
306  Sherman      M  5939 Garcia Forges Suite 297  San Antonio  ...  29.3641   

        long city_pop   job         dob                         trans_num  \
306 -98.4924  1595797  Land  1995-10-17  0e78856930bfd4999ecff99c7f0fb2c7   

      unix_time  merch_lat merch_long is_fraud  
306  1378556845  28.764426 -99.000738        0  

[1 rows x 22 columns]


🕵️ Fact Checker result:
📋 Fact Checker Report:
⚠️ Z-score anomaly in: city_pop
⚠️ IQR outlier in: city_pop


# 🧠 Agent Debate System

This section simulates a legal-style debate between multiple LLM-powered agents:

- **Prosecutor**: Argues that the transaction is fraudulent
- **Defender**: Argues that it is legitimate
- **Fact Checker**: Provides neutral statistical insight (already defined)
- **Judge**: Reviews all arguments and issues a verdict

Agents use information about the test transaction and its top precedent neighbors to frame their arguments. The judge sees only the fact-checker's report.


In [44]:
# === 💬 Prompt Configuration Block ===

# System Role Description
SYSTEM_PROMPT = """You are a legal debate system simulating a fraud trial. There are three roles:
- The Prosecutor argues that the transaction is fraudulent.
- The Defender argues that the transaction is not fraudulent.
- A Judge listens to both sides and makes a decision based on the arguments and the Fact Checker’s report.

Each side will take turns making their case. Use structured, bullet-point reasoning, and address your opponent’s key claims directly.
"""

# Prompt given to each debating role in each round
ROLE_PROMPT_TEMPLATE = """{role}: Present your argument in 3–5 concise bullet points.
- Be direct and specific.
- Avoid long sentences or fluff.
- Refute at least one claim from the opposing side if possible.
"""

# Judge's Decision Prompt — FRAUDTEST domain (clean and natural)
JUDGE_PROMPT = """Judge: Based on the debate and fact checking, what is your verdict?
(Answer with: FRAUD or NOT FRAUD and explain your reasoning in bullet points.)

Your role is to weigh all the evidence fairly and clearly.

Consider:
- The strength and specificity of the Prosecutor’s arguments
- The clarity and persuasiveness of the Defender’s rebuttal
- What the Fact Checker report reveals
- Whether the transaction matches a known pattern of fraud in the precedent cases

If the case strongly resembles past fraud or the Prosecutor makes a compelling case that is not adequately refuted, you should vote FRAUD.

If the Defender raises reasonable doubt and the transaction seems typical or explainable, you should vote NOT FRAUD.

Be decisive. Summarize your reasoning in 3–5 clear bullet points.
"""


In [41]:
def openai_iterative_debate_with_judge(
    row,
    train_df,
    index,
    scaler,
    model,
    numeric_cols,
    text_cols,
    client,
    deployment_name,
    domain,
    max_rounds=2,
    verbose=True,
    log_to_json=False,
    system_prompt=None,
    role_prompt_template=None,
    judge_prompt=None,
):
    import json
    import uuid

    # === Default to global prompt config if not supplied ===
    if system_prompt is None:
        system_prompt = SYSTEM_PROMPT
    if role_prompt_template is None:
        role_prompt_template = ROLE_PROMPT_TEMPLATE
    if judge_prompt is None:
        judge_prompt = JUDGE_PROMPT

    # === Format transaction info ===
    def format_transaction_for_prompt(row, domain):
        label_cols = {
            "insurance": "insuranceclaim",
            "credit": "Class",
            "fraudtest": "is_fraud"
        }
        exclude_col = label_cols.get(domain)
        return ", ".join(
            f"{col}={round(val, 2) if isinstance(val, (int, float)) else val}"
            for col, val in row.items()
            if col != exclude_col
        )

    transaction_str = format_transaction_for_prompt(row, domain)

    # === Retrieve legal precedents ===
    neighbors_df = find_nearest_neighbors(row, train_df, index, scaler, model, numeric_cols, text_cols, k=5)
    legal_precedents = format_neighbors_generic(neighbors_df)

    # === Run Fact Checker ===
    fact_report = generate_fact_checker_report(row, train_df, domain)

    # === Initialize debate log ===
    debate_log = {
        "transaction": transaction_str,
        "precedents": legal_precedents,
        "fact_checker": fact_report,
        "rounds": [],
        "verdict": None,
        "ground_truth": None,
        "judge_correct": None,
        "outcome_type": None
    }

    # === Compose initial messages ===
    debate_intro = f"""Transaction under investigation:

{transaction_str}

Here are similar precedent transactions (including ground truth labels):

{legal_precedents}
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": debate_intro}
    ]

    if verbose:
        print("🧠 Beginning debate...")
        print("🦾 Transaction:\n", transaction_str)
        print("\n📚 Legal precedents:\n", legal_precedents)
        print("\n🎯 Fact Checker (visible only to judge):\n", fact_report)
        print("-" * 60)

    # === Alternate rounds ===
    for round_num in range(max_rounds):
        role = "Prosecutor" if round_num % 2 == 0 else "Defender"
        prompt = role_prompt_template.format(role=role)

        if verbose:
            print(f"\n⚖️ Round {round_num + 1} - {role}:")

        messages.append({"role": "user", "content": prompt})

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            temperature=0.7,
            max_tokens=500
        )
        reply = response.choices[0].message.content.strip()
        messages.append({"role": "assistant", "content": reply})
        debate_log["rounds"].append({role: reply})

        if verbose:
            print(reply)

    # === Add Fact Checker report (for judge only) ===
    messages.append({"role": "user", "content": f"Fact Checker report: {fact_report}"})

    # === Add Judge Prompt ===
    messages.append({"role": "user", "content": judge_prompt})

    final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        temperature=0.3,
        max_tokens=600
    )

    judge_reply = final_response.choices[0].message.content.strip()
    debate_log["verdict"] = judge_reply

    # === Evaluate against ground truth ===
    truth = row.get("is_fraud", row.get("Class", row.get("insuranceclaim", "unknown")))
    is_fraud_truth = (truth == 1)

    verdict_upper = judge_reply.upper()
    if "NOT FRAUD" in verdict_upper:
        judge_says_fraud = False
    elif "FRAUD" in verdict_upper:
        judge_says_fraud = True
    else:
        judge_says_fraud = None  # Optional: raise warning or log

    is_judge_right = (judge_says_fraud == is_fraud_truth)

    # Store outcome label (TP, FP, TN, FN)
    if is_fraud_truth and judge_says_fraud:
        outcome = "TP"
    elif not is_fraud_truth and not judge_says_fraud:
        outcome = "TN"
    elif not is_fraud_truth and judge_says_fraud:
        outcome = "FP"
    elif is_fraud_truth and not judge_says_fraud:
        outcome = "FN"
    else:
        outcome = "??"

    debate_log["ground_truth"] = "FRAUD" if is_fraud_truth else "NOT FRAUD"
    debate_log["judge_correct"] = is_judge_right
    debate_log["outcome_type"] = outcome

    if verbose:
        print("\n👩‍⚖️ Final Verdict:")
        print(judge_reply)
        print(f"\n📊 Ground Truth Label: {debate_log['ground_truth']}")
        print(f"📌 Outcome: {outcome}")
        print("✅ Judge's verdict matches the ground truth." if is_judge_right else "❌ Judge's verdict does NOT match the ground truth.")

    # === Optionally log to JSON ===
    if log_to_json:
        uid = str(uuid.uuid4())[:8]
        with open(f"debate_log_{uid}.json", "w") as f:
            json.dump(debate_log, f, indent=2)

    return judge_reply, transaction_str, legal_precedents, fact_report, is_judge_right, outcome


In [54]:
# ================================================
# 🧪 One-Off Sample Sanity Check (Verbose = True)
# ================================================

sample_test_row = test_df.sample(1).iloc[0]

verdict, transaction_str, precedents, fact_report, is_judge_right, error_type = openai_iterative_debate_with_judge(
    row=sample_test_row,
    train_df=train_df,
    index=index,
    scaler=scaler,
    model=model,
    numeric_cols=numeric_cols,
    text_cols=text_cols,
    client=client,
    deployment_name=deployment_name,
    domain=domain,
    max_rounds=2,
    verbose=True
)

# Infer ground truth and model prediction from verdict
truth = sample_test_row.get("is_fraud", sample_test_row.get("Class", sample_test_row.get("insuranceclaim", "unknown")))


🧠 Beginning debate...
🦾 Transaction:
 trans_date_trans_time=2020-11-12 23:48:17, cc_num=30242921189660, merchant=Bernier, Volkman and Hoeger, category=misc_net, amt=665.71, first=Dalton, last=Jones, gender=M, street=3304 Hernandez Run, city=Afton, state=MI, zip=49705, lat=45.36, long=-84.47, city_pop=937, job=Technical brewer, dob=1958-04-06, trans_num=017d7358ab39ae4ce9c39647d708600f, unix_time=1384300097, merch_lat=45.07, merch_long=-84.68

📚 Legal precedents:
 Neighbor 1:
Distance: 0.037; trans_date_trans_time: 2020-11-12 22:27:23; cc_num: 30242921189660; merchant: Bernier, Volkman and Hoeger; category: misc_net; amt: 684.72; first: Dalton; last: Jones; gender: M; street: 3304 Hernandez Run; city: Afton; state: MI; zip: 49705; lat: 45.3637; long: -84.4695; city_pop: 937; job: Technical brewer; dob: 1958-04-06; trans_num: b34bbc5d5970e6b8643452db53338d5b; unix_time: 1384295243; merch_lat: 45.802949; merch_long: -85.301461; is_fraud: 1

Neighbor 2:
Distance: 0.573; trans_date_trans_ti

In [95]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# === Tunable Parameters ===
n_samples = 100
verbose = False

# === Outcome Counters ===
tp = tn = fp = fn = 0

# === Track outcomes for optional detailed inspection ===
outcomes = []

# === Run Mock Trial on n_samples ===
for i in range(n_samples):
    row = test_df.sample(1).iloc[0]

    # Run the agent system – must now return error_type (e.g. "TP", "FP", etc.)
    verdict, transaction_str, precedents, fact_report, is_judge_right, error_type = openai_iterative_debate_with_judge(
        row=row,
        train_df=train_df,
        index=index,
        scaler=scaler,
        model=model,
        numeric_cols=numeric_cols,
        text_cols=text_cols,
        client=client,
        deployment_name=deployment_name,
        domain=domain,
        max_rounds=1,
        verbose=verbose
    )

    # Count outcomes
    if error_type == "TP":
        tp += 1
    elif error_type == "TN":
        tn += 1
    elif error_type == "FP":
        fp += 1
    elif error_type == "FN":
        fn += 1

    # Track per-case data
    truth = row.get("is_fraud", row.get("Class", row.get("insuranceclaim", "unknown")))
    is_fraud_truth = (truth == 1)
    verdict_upper = verdict.upper()
    is_fraud_pred = "NOT FRAUD" not in verdict_upper and "FRAUD" in verdict_upper

    outcomes.append({
        "outcome": error_type,
        "truth": "FRAUD" if is_fraud_truth else "NOT FRAUD",
        "verdict": "FRAUD" if is_fraud_pred else "NOT FRAUD",
        "is_correct": is_judge_right
    })

# === Results Table ===
results_df = pd.DataFrame(outcomes)
summary_df = pd.DataFrame({
    "Outcome Type": ["True Positive", "True Negative", "False Positive", "False Negative"],
    "Count": [tp, tn, fp, fn]
})

display(summary_df)

# === Plot ===
fig, ax = plt.subplots()
labels = ["TP", "TN", "FP", "FN"]
sizes = [tp, tn, fp, fn]
colors = ["#4CAF50", "#2196F3", "#FFC107", "#F44336"]
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
ax.axis('equal')
plt.title(f"Debate Outcome Distribution (n={n_samples})")
plt.show()


Outcome Type  Count
0   True Positive     32
1   True Negative     50
2  False Positive      6
3  False Negative     12

## 🌱 Future Directions and Tabled Ideas for Improvement

These ideas were identified as promising but intentionally deferred until a later stage of development:

- **Pattern Evidence Summary Injection:** Before the judge delivers a verdict, inject a concise, structured summary of key fraud indicators derived from precedent transactions (e.g., same cardholder, time proximity, similar amounts, geographic deviation). This provides the model with an evidence-based checklist that reinforces consistent and accurate rulings without relying solely on the quality of the debate text.

- **Hallucination Detection:** Automatically flag and analyze when the prosecutor or defender invents facts not grounded in the case.

- **Automatic Verdict Summarization:** Simplify the judge’s prose to highlight only the top 2–3 reasons for the ruling.

- **Fact-Checker Risk Score:** Create a numerical anomaly risk score based on the Fact-Checker report for statistical calibration.

- **Agent Retraining via RAG + Supervised Finetuning:** Use misclassified or high-risk cases to retrain agent behavior with reinforcement learning or distillation.

- **Agent Role Scorecarding:** Track per-agent precision/recall and use performance to guide strategic prompting or agent self-improvement.

- **Dynamic Prompt Calibration:** Automatically adjust prompt structure and verbosity to reduce LLM cutoff and hallucination risks.

- **Error-Driven Curriculum:** Weight harder, previously misclassified examples higher in future training cycles or testing.

These will be explored once the base system proves robust through initial testing and error analysis.
